In [1]:
""" EXPLANATION FOR QUESTION1"""
"""Processing Steps
Setup Environment:

Imports required libraries (cv2, numpy, mediapipe, etc.).
Initializes a MediaPipe face mesh model (though it's unused in the script).
Defines an output folder (morph_results1) and ensures it starts fresh by deleting any existing contents.
Load Input Data:

Reads source and target images (source.jpg and target.jpg).
Extracts tie points from tie_points.txt, which contains manually selected corresponding feature points between both images.
Adds four corner points of the images to stabilize morphing.
Perform Delaunay Triangulation:

Computes Delaunay triangulation on the source image’s tie points to get a mesh of non-overlapping triangles.
Uses the same triangulation structure for both images to ensure smooth transitions.
Generate Morphing Sequence:

Iterates through alpha values from 0 to 1 in steps (num_frames = 20).
Computes interpolated tie points using (1 - alpha) * points1 + alpha * points2.
Creates a blank canvas and morphs each triangle individually using:
Affine transformations (to warp triangles).
Alpha blending (to smoothly transition between images).
Save Output Frames:

Writes each intermediate frame (morph_00.jpg, morph_01.jpg, etc.) to morph_results1/.
Generate GIF Animation:

Reads the saved frames and converts them into a GIF animation (morph_animation_q1.gif).
Uses imageio.mimsave() to create the final output.
Input Requirements & Checks
Image Compatibility:

Both images must have the same dimensions (height & width).
If not, pre-resizing is needed to prevent distortion.
Tie Points File (tie_points.txt):

Must be correctly formatted:
The user can change the number of frames and the time for which the frames are displayed in the gif
First line: Number of tie points (T).
Next T lines: Four space-separated integers representing corresponding (x, y) coordinates in both images.
If incorrectly formatted or missing data, the script will crash.
Delaunay Triangulation Validity:

If tie points are not well distributed, bad triangulation can occur, leading to warped results.
Folder Structure:

The script deletes old outputs before running.
Ensure no important files are stored in morph_results_1/, as they will be lost.
IT TAKES A FEW SECONDS TO SAVE THE FILE AND THEN THE RESULT IS DISPLAYED ALONG WITH THE ADDRESS"""

' EXPLANATION FOR QUESTION1'

In [5]:
#Q1, where tie points would be provided
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import Delaunay
import glob
import imageio
import os
import shutil

OUTPUT_FOLDER = "morph_results_1"
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True)

if os.path.exists(OUTPUT_FOLDER):
    shutil.rmtree(OUTPUT_FOLDER)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

def load_tie_points(file_path, img_width, img_height):
    """Load tie points from a file and add four corner points."""
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    T = int(lines[0].strip())  # Number of tie points
    points1, points2 = [], []
    
    for line in lines[1:T + 1]:
        x1a, y1a, x1b, y1b = map(int, line.strip().split())
        points1.append((x1a, y1a))
        points2.append((x1b, y1b))
    
    # Add four corner points
    corners = [(0, 0), (img_width - 1, 0), (img_width - 1, img_height - 1), (0, img_height - 1)]
    points1.extend(corners)
    points2.extend(corners)
    
    return np.array(points1, np.float32), np.array(points2, np.float32)

def apply_affine_transform(src, src_tri, dst_tri, size):
    """Apply an affine transform to warp a triangle with border reflection."""
    warp_mat = cv2.getAffineTransform(np.float32(src_tri), np.float32(dst_tri))
    dst = cv2.warpAffine(src, warp_mat, (size[0], size[1]), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)
    return dst

def morph_triangle(img1, img2, img_morph, tri1, tri2, tri_morph, alpha):
    """Morph a single triangle using affine transforms and alpha blending."""
    r1 = cv2.boundingRect(np.float32(tri1))
    r2 = cv2.boundingRect(np.float32(tri2))
    r_morph = cv2.boundingRect(np.float32(tri_morph))

    tri1_cropped = [(p[0] - r1[0], p[1] - r1[1]) for p in tri1]
    tri2_cropped = [(p[0] - r2[0], p[1] - r2[1]) for p in tri2]
    tri_morph_cropped = [(p[0] - r_morph[0], p[1] - r_morph[1]) for p in tri_morph]

    img1_cropped = img1[r1[1]:r1[1]+r1[3], r1[0]:r1[0]+r1[2]]
    img2_cropped = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]]

    warp1 = apply_affine_transform(img1_cropped, tri1_cropped, tri_morph_cropped, (r_morph[2], r_morph[3]))
    warp2 = apply_affine_transform(img2_cropped, tri2_cropped, tri_morph_cropped, (r_morph[2], r_morph[3]))

    warp_morphed = cv2.addWeighted(warp1, 1 - alpha, warp2, alpha, 0)
    mask = np.zeros((r_morph[3], r_morph[2]), dtype=np.uint8)
    cv2.fillConvexPoly(mask, np.int32(tri_morph_cropped), 255)

    roi = img_morph[r_morph[1]:r_morph[1]+r_morph[3], r_morph[0]:r_morph[0]+r_morph[2]]
    roi_masked = cv2.bitwise_and(roi, roi, mask=cv2.bitwise_not(mask))
    warp_morphed_masked = cv2.bitwise_and(warp_morphed, warp_morphed, mask=mask)
    img_morph[r_morph[1]:r_morph[1]+r_morph[3], r_morph[0]:r_morph[0]+r_morph[2]] = roi_masked + warp_morphed_masked

def generate_morph_sequence(img1, img2, points1, points2, triangles, num_frames):
    """Generate morph frames from 0 to 1 in `num_frames` steps."""
    filenames = []
    for i in range(num_frames + 1):
        alpha = i / num_frames
        points_morph = (1 - alpha) * points1 + alpha * points2
        img_morph = np.zeros_like(img1, dtype=np.uint8)

        for tri_index in triangles:
            tri1 = [points1[idx] for idx in tri_index]
            tri2 = [points2[idx] for idx in tri_index]
            tri_morph = [points_morph[idx] for idx in tri_index]
            morph_triangle(img1, img2, img_morph, tri1, tri2, tri_morph, alpha)

        filename = os.path.join(OUTPUT_FOLDER, f"morph_{i:02d}.jpg")
        cv2.imwrite(filename, img_morph)
        filenames.append(filename)
    
    create_gif_from_saved_images()

def create_gif_from_saved_images():
    """Create a GIF from saved morph images."""
    filenames = sorted(glob.glob(os.path.join(OUTPUT_FOLDER, "morph_*.jpg")))
    images = [cv2.imread(f) for f in filenames if cv2.imread(f) is not None]
    images = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in images]

    if images:
        gif_filename = os.path.join(OUTPUT_FOLDER, "morph_animation_q1.gif")
        imageio.mimsave(gif_filename, images, format='GIF', duration=0.5)
        print(f"GIF saved as {gif_filename}")
    else:
        print("No valid images found for GIF creation.")

def main():
    img1 = cv2.imread("source.jpg")
    img2 = cv2.imread("target.jpg")
    tiepoints_file = "tie_points.txt"

    if img1 is None or img2 is None:
        raise ValueError("One or both images could not be loaded. Check paths.")
    h1, w1, _ = img1.shape
    h2, w2, _ = img2.shape
    
    if h1 * w1 < h2 * w2:
        img1 = cv2.resize(img1, (w2, h2))
    else:
        img2 = cv2.resize(img2, (w1, h1))

    img_height, img_width = img1.shape[:2]
    points1, points2 = load_tie_points(tiepoints_file, img_width, img_height)
    delaunay = Delaunay(points1)
    triangles = delaunay.simplices

    num_frames = 20  # Default frame count
    generate_morph_sequence(img1, img2, points1, points2, triangles, num_frames)

if __name__ == "__main__":
    main()


GIF saved as morph_results_1\morph_animation_q1.gif


In [ ]:
#Explanation for Q2
"""Processing Steps
Setup Environment:

Imports required libraries (cv2, numpy, mediapipe, etc.).
Initializes a MediaPipe Face Mesh model for landmark extraction.
Defines an output folder (morph_results), deleting previous outputs to avoid conflicts.
Extract Facial Landmarks:

Uses MediaPipe to detect 468 face landmarks from each input image.
Adds 8 boundary points (image corners + edge midpoints) to ensure proper morphing.
Converts the landmark data into NumPy arrays for further processing.
Perform Delaunay Triangulation:

Computes Delaunay triangulation on the first image's landmark points.
Uses the same triangulation structure for both images to maintain consistency.
Save Visualizations:

Saves landmark overlay images (source_landmarks.jpg, target_landmarks.jpg).
Saves Delaunay triangulation images (source_triangulation.jpg, target_triangulation.jpg) for debugging.
Generate Morphing Sequence:

Iterates through alpha values from 0 to 1 in num_frames steps.
Computes interpolated tie points using (1 - alpha) * points1 + alpha * points2.
Morphs each Delaunay triangle individually:
Applies affine transformations to warp the triangles.
Blends the warped triangles using alpha blending.
Save Output Frames:

Writes each intermediate frame (morph_00.jpg, morph_01.jpg, etc.) to morph_results/.
Create GIF Animation:

Reads the saved frames and converts them into a GIF animation (morph_animation_q2.gif).
Along with the gif file, the landmarks points marked on both the photos are also saved. Also, the same for the triangulation image.
Ensures all frames have the same resolution to prevent GIF corruption.
Uses imageio.mimsave() to generate the final output.
Input Requirements & Checks
Image Compatibility:

Both images must have the same dimensions (height & width).
If not, the script resizes the smaller image to match the larger one.
Face Detection:

The script requires a visible face in both images.
If no face is detected, an error is raised (ValueError: No face detected!).
Delaunay Triangulation Validity:

If landmarks are not well-distributed, it may cause triangle distortions.
Additional boundary points help stabilize the triangulation.
Folder Structure:

The script deletes old outputs before running.
Ensure no important files are stored in morph_results_2/, as they will be lost.
IT TAKES A FEW SECONDS TO SAVE THE FILE AND THEN THE RESULT IS DISPLAYED ALONG WITH THE ADDRESS
IN CASE, THE FACE IS NOT DETECTED, TRY GIVING A CROPPED VERESION OF THE IMAGE"""

In [4]:
#Q2, no tie points are provided, images are directly converted
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import Delaunay
import glob
import imageio
import os
import shutil

OUTPUT_FOLDER = "morph_results_2"
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, 
                                  max_num_faces=1, 
                                  refine_landmarks=True)

if os.path.exists(OUTPUT_FOLDER):
    shutil.rmtree(OUTPUT_FOLDER)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

def get_landmarks(image):
    """Extract all 468 MediaPipe face landmarks + 8 boundary points."""
    h, w, _ = image.shape
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if not results.multi_face_landmarks:
        raise ValueError("No face detected!")
    
    # All 468 landmarks
    landmarks = [(int(results.multi_face_landmarks[0].landmark[i].x * w),
                  int(results.multi_face_landmarks[0].landmark[i].y * h)) 
                 for i in range(468)]
    
    # Add 8 boundary points (corners + mid-edges)
    boundary_points = [(0, 0), (w//4, 0), (w//2, 0), (3*w//4, 0), (w-1, 0), 
        (w-1, h//4), (w-1, h//2), (w-1, 3*h//4), (w-1, h-1),
        (3*w//4, h-1), (w//2, h-1), (w//4, h-1), (0, h-1),
        (0, 3*h//4), (0, h//2), (0, h//4)]
    return np.array(landmarks + boundary_points, np.float32)

def apply_affine_transform(src, src_tri, dst_tri, size):
    """Apply an affine transform to warp a triangle with border reflection."""
    warp_mat = cv2.getAffineTransform(np.float32(src_tri), np.float32(dst_tri))
    # Use BORDER_REFLECT_101 to avoid black edges at the boundary
    dst = cv2.warpAffine(
        src,
        warp_mat,
        (size[0], size[1]),
        None,
        flags=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_REFLECT_101
    )
    return dst
def save_landmark_image(image, points, filename):
    """Draw landmarks on image and save."""
    img_copy = image.copy()
    for x, y in points:
        cv2.circle(img_copy, (int(x), int(y)), 2, (0, 255, 0), -1)
    cv2.imwrite(os.path.join(OUTPUT_FOLDER, filename), img_copy)

def save_delaunay_image(image, points, triangles, filename):
    """Draw Delaunay triangulation on image and save."""
    img_copy = image.copy()
    for tri in triangles:
        pts = [tuple(map(int, points[i])) for i in tri]
        cv2.line(img_copy, pts[0], pts[1], (255, 0, 0), 1)
        cv2.line(img_copy, pts[1], pts[2], (255, 0, 0), 1)
        cv2.line(img_copy, pts[2], pts[0], (255, 0, 0), 1)
    cv2.imwrite(os.path.join(OUTPUT_FOLDER, filename), img_copy)

def clip_rect(rect, w_img, h_img):
    """Ensure the rectangle is within the image boundaries."""
    x, y, w, h = rect
    if x < 0:
        x = 0
    if y < 0:
        y = 0
    if x + w > w_img:
        w = w_img - x
    if y + h > h_img:
        h = h_img - y
    return (x, y, w, h)

def morph_triangle(img1, img2, img_morph, tri1, tri2, tri_morph, alpha):
    """Morph a single triangle using affine transforms and alpha blending."""
    # Compute bounding rectangles
    r1 = cv2.boundingRect(np.float32(tri1))
    r2 = cv2.boundingRect(np.float32(tri2))
    r_morph = cv2.boundingRect(np.float32(tri_morph))

    # Clip them so they don't go out-of-bounds
    r1 = clip_rect(r1, img1.shape[1], img1.shape[0])
    r2 = clip_rect(r2, img2.shape[1], img2.shape[0])
    r_morph = clip_rect(r_morph, img_morph.shape[1], img_morph.shape[0])

    # If any rect is effectively empty, skip
    if r1[2] <= 0 or r1[3] <= 0 or r2[2] <= 0 or r2[3] <= 0 or r_morph[2] <= 0 or r_morph[3] <= 0:
        return

    # Offset the triangle coordinates relative to cropped regions
    tri1_cropped = [(p[0] - r1[0], p[1] - r1[1]) for p in tri1]
    tri2_cropped = [(p[0] - r2[0], p[1] - r2[1]) for p in tri2]
    tri_morph_cropped = [(p[0] - r_morph[0], p[1] - r_morph[1]) for p in tri_morph]

    # Extract the region of interest from each image
    img1_cropped = img1[r1[1]:r1[1]+r1[3], r1[0]:r1[0]+r1[2]]
    img2_cropped = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]]

    # Warp both triangle patches into the morphed triangle shape
    warp1 = apply_affine_transform(
        img1_cropped, 
        tri1_cropped, 
        tri_morph_cropped, 
        (r_morph[2], r_morph[3])
    )
    warp2 = apply_affine_transform(
        img2_cropped, 
        tri2_cropped, 
        tri_morph_cropped, 
        (r_morph[2], r_morph[3])
    )

    # Blend the warped triangles
    warp_morphed = cv2.addWeighted(warp1, 1 - alpha, warp2, alpha, 0)

    # Create a mask for the triangle region
    mask = np.zeros((r_morph[3], r_morph[2]), dtype=np.uint8)
    cv2.fillConvexPoly(mask, np.int32(tri_morph_cropped), 255)

    # Place the morphed triangle in its position within img_morph
    roi = img_morph[r_morph[1]:r_morph[1]+r_morph[3], r_morph[0]:r_morph[0]+r_morph[2]]
    roi_masked = cv2.bitwise_and(roi, roi, mask=cv2.bitwise_not(mask))
    warp_morphed_masked = cv2.bitwise_and(warp_morphed, warp_morphed, mask=mask)
    img_morph[r_morph[1]:r_morph[1]+r_morph[3], r_morph[0]:r_morph[0]+r_morph[2]] = roi_masked + warp_morphed_masked

def generate_morph_sequence(img1, img2, points1, points2, triangles, num_frames):
    """Generate morph frames from 0 to 1 in `num_frames` steps."""
    filenames = []
    for i in range(num_frames + 1):
        alpha = i / num_frames
        points_morph = (1 - alpha) * points1 + alpha * points2
        img_morph = np.zeros_like(img1, dtype=np.uint8)

        for tri_index in triangles:
            tri1 = [points1[idx] for idx in tri_index]
            tri2 = [points2[idx] for idx in tri_index]
            tri_morph = [points_morph[idx] for idx in tri_index]
            morph_triangle(img1, img2, img_morph, tri1, tri2, tri_morph, alpha)

        filename = os.path.join(OUTPUT_FOLDER, f"morph_{i:02d}.jpg")
        cv2.imwrite(filename, img_morph)
        filenames.append(filename)
    
    create_gif_from_saved_images()

def create_gif_from_saved_images():
    """Create a GIF from already saved morph images."""
    filenames = sorted(glob.glob(os.path.join(OUTPUT_FOLDER, "morph_*.jpg")))
    images = []
    
    # Read the first image to get the target shape
    if filenames:
        first_img = cv2.imread(filenames[0])
        if first_img is None:
            print("Error: Could not load first image for GIF creation.")
            return
        h, w, _ = first_img.shape  # Reference shape
    
    for f in filenames:
        img = cv2.imread(f)
        if img is not None:
            # Resize images to ensure consistent dimensions
            img_resized = cv2.resize(img, (w, h))
            images.append(cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB))
        else:
            print(f"Warning: Could not read {f}, skipping.")

    if images:
        gif_filename = os.path.join(OUTPUT_FOLDER, "morph_animation_q2.gif")
        imageio.mimsave(gif_filename, images, format='GIF', duration=0.5)
        print(f"GIF saved as {gif_filename}")
    else:
        print("No valid images found for GIF creation.")


def main():
    # Load images (make sure they have same height and width or resize first)
    img1 = cv2.imread("source.jpg")
    # Instead of reading a separate target image from disk, we create it:
    img2 = cv2.imread("target.jpg")

    if img1 is None or img2 is None:
        raise ValueError("One or both images could not be loaded. Check paths.")
    h1, w1, _ = img1.shape
    h2, w2, _ = img2.shape
    
    if h1 * w1 < h2 * w2:
        img1 = cv2.resize(img1, (w2, h2))
    else:
        img2 = cv2.resize(img2, (w1, h1))
    
    # Optionally, ensure both images are same size:
    # img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))

    points1 = get_landmarks(img1)
    points2 = get_landmarks(img2)

    # Save landmarks visualization
    save_landmark_image(img1, points1, "source_landmarks.jpg")
    save_landmark_image(img2, points2, "target_landmarks.jpg")


    # Create a Delaunay triangulation on the first set of points
    # (Alternatively, you could triangulate an "average shape".)
    delaunay = Delaunay(points1)
    triangles = delaunay.simplices
    # Save Delaunay triangulation visualization
    save_delaunay_image(img1, points1, triangles, "source_triangulation.jpg")
    save_delaunay_image(img2, points2, triangles, "target_triangulation.jpg")


    # Morph!
    try:
        num_frames = int(input("Enter the number of morph frames: "))
        if num_frames <= 20:
            raise ValueError
    except ValueError:
        print("Invalid input, using default 20 frames.")
        num_frames = 20
    generate_morph_sequence(img1, img2, points1, points2, triangles, num_frames)

    

if __name__ == "__main__":
    main()


Enter the number of morph frames:  50


GIF saved as morph_results_2\morph_animation_q2.gif
